In [5]:
import torch_xla.runtime

x = torch.randn(2, 3, 4, 5, requires_grad=True)
y = torch.randn(2, 5, 6, requires_grad=True)

with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    x = x.to('xla').requires_grad_()
    y = y.to('xla').requires_grad_()
    out = torch.einsum("ebcm,emh->ebch", x, y)

print(torch_xla._XLAC._get_xla_tensors_text([out]))
print(torch_xla._XLAC._get_xla_tensors_hlo([out]))

IR {
  %0 = f32[2,5,6]{2,1,0} xla::device_data(), xla_shape=f32[2,5,6]{2,1,0}
  %1 = f32[2,3,4,5]{3,2,1,0} xla::device_data(), xla_shape=f32[2,3,4,5]{3,2,1,0}
  %2 = f32[2,3,4,6]{3,2,1,0} aten::einsum(%1, %0), xla_shape=f32[2,3,4,6]{3,2,1,0}, ROOT=0
}

HloModule IrToHlo.6, entry_computation_layout={(f32[2,5,6]{2,1,0}, f32[2,3,4,5]{3,2,1,0})->(f32[2,3,4,6]{3,2,1,0})}

ENTRY %IrToHlo.6 (p0.1: f32[2,5,6], p1.2: f32[2,3,4,5]) -> (f32[2,3,4,6]) {
  %p1.2 = f32[2,3,4,5]{3,2,1,0} parameter(1)
  %p0.1 = f32[2,5,6]{2,1,0} parameter(0)
  %dot.3 = f32[2,3,4,6]{3,2,1,0} dot(f32[2,3,4,5]{3,2,1,0} %p1.2, f32[2,5,6]{2,1,0} %p0.1), lhs_batch_dims={0}, lhs_contracting_dims={3}, rhs_batch_dims={0}, rhs_contracting_dims={1}, frontend_attributes={grad_x="false",grad_y="false"}
  %transpose.4 = f32[2,3,4,6]{3,2,1,0} transpose(f32[2,3,4,6]{3,2,1,0} %dot.3), dimensions={0,1,2,3}
  ROOT %tuple.5 = (f32[2,3,4,6]{3,2,1,0}) tuple(f32[2,3,4,6]{3,2,1,0} %transpose.4)
}


